In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')


def linear_fit(x, a, b):
    
    return a + b * x



# load data

In [ ]:
os.listdir(os.path.join(DATAPATH))

In [ ]:
os.listdir(os.path.join(RESULTPATH))

In [ ]:
#test_data = mf.data.MFDataset(os.path.join(DATAPATH, '210930_87_18595_energy_angle_test_slice8192.h5'))
template_data = mf.data.MFDataset(os.path.join(DATAPATH, '211027_84_25_2cm.h5'))

test_data = mf.data.MFDataset(os.path.join(DATAPATH, '211106_84_2cm_low_energy.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

In [ ]:
template_metadata = pd.DataFrame(template_data.metadata)
test_metadata = pd.DataFrame(test_data.metadata)
#test_metadata = pd.DataFrame(test_data.metadata)

# pick signal, compute scores

In [ ]:
#signal_angle = 86.5
#signal_energy = 18587.5
#var = 1.38e-23 * 10 * 50 * 200e6

#signal_index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]
#print(signal_index)

#signal = template_data.data[signal_index, :]
#norm = 1 / np.sqrt(var * np.vdot(signal, signal))
#template = signal * norm


#signals = test_data.data[:]
#templates = template_data.data[:]

nsignal = test_data.data.shape[0]
ntemplate = template_data.data.shape[0]
nsplit = 16

template_indx_list = np.array_split(np.arange(0, ntemplate, 1), nsplit)
signal_indx_list = np.array_split(np.arange(0, nsignal, 1), nsplit)

scores = np.zeros((nsignal, ntemplate))

nloops = nsplit ** 2

counter = 0
for i, indxs1 in enumerate(signal_indx_list):
    for j, indxs2 in enumerate(template_indx_list):
        #print(indxs1, indxs2)
        #input()
        #rows, cols = np.meshgrid(indxs1, indxs2)
        scores[indxs1[0]:indxs1[-1] + 1, indxs2[0]:indxs2[-1] + 1] = abs(np.matmul(test_data.data[indxs1, :], template_data.data[indxs2, :].conjugate().T))
        
        counter += 1
        if counter % 5 == 4:
            print(f'{counter + 1} / {nloops}')
        
            
np.save(os.path.join(RESULTPATH, 'scores', '211106_84_25_2cm_low_energy_on_template'), scores)


In [ ]:

fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(1,1,1)

ax.imshow(scores, interpolation='none')



In [ ]:
sym_scores = np.zeros(scores.shape)
for i in range(scores.shape[0]):
    for j in range(scores.shape[1]):
        if i <= j:
            sym_scores[i, j] = scores[i, j]
            sym_scores[j, i] = scores[i, j]
            

In [ ]:
fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(1,1,1)

ax.imshow(sym_scores, interpolation='none')

In [ ]:
np.save(os.path.join(RESULTPATH, 'scores', '211106_84_25_2cm_random_on_template'), sym_scores)